In [14]:
import os

if os.path.exists("bench") == False:
    os.mkdir("bench")

!wget https://arxiv.org/pdf/1706.03762 -O bench/attention.pdf
!wget https://arxiv.org/pdf/2407.01449 -O bench/colpali.pdf
!wget https://arxiv.org/pdf/2310.06825 -O bench/mistral.pdf


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


--2024-10-10 20:53:22--  https://arxiv.org/pdf/1706.03762
Resolving arxiv.org (arxiv.org)... 151.101.67.42, 151.101.131.42, 151.101.3.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.67.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2215244 (2.1M) [application/pdf]
Saving to: ‘bench/attention.pdf’

bench/attention.pdf 100%[===================>]   2.11M  1.64MB/s    in 1.3s    

2024-10-10 20:53:23 (1.64 MB/s) - ‘bench/attention.pdf’ saved [2215244/2215244]

--2024-10-10 20:53:23--  https://arxiv.org/pdf/2407.01449
Resolving arxiv.org (arxiv.org)... 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


151.101.195.42, 151.101.3.42, 151.101.131.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.195.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9380119 (8.9M) [application/pdf]
Saving to: ‘bench/colpali.pdf’

bench/colpali.pdf   100%[===================>]   8.95M  2.70MB/s    in 3.3s    

2024-10-10 20:53:27 (2.70 MB/s) - ‘bench/colpali.pdf’ saved [9380119/9380119]

--2024-10-10 20:53:27--  https://arxiv.org/pdf/2310.06825
Resolving arxiv.org (arxiv.org)... 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


151.101.67.42, 151.101.195.42, 151.101.3.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.67.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3749788 (3.6M) [application/pdf]
Saving to: ‘bench/mistral.pdf’

bench/mistral.pdf   100%[===================>]   3.58M  2.88MB/s    in 1.2s    

2024-10-10 20:53:29 (2.88 MB/s) - ‘bench/mistral.pdf’ saved [3749788/3749788]



In [15]:
# from sentence_transformers import SentenceTransformer
# import pymupdf
# from semantic_text_splitter import TextSplitter
# import time
# splitter = TextSplitter(1000)
# files = []
# model = SentenceTransformer('BAAI/bge-small-en-v1.5')

# # get all pdfs from test_files
# now = time.time()

# for file in os.listdir("bench"):

#   text = []
#   doc = pymupdf.open("bench/" + file)

#   for page in doc:
#       text.append(page.get_text())

#   text = " ".join(text)
#   chunks = splitter.chunks(text)
#   embeddings = model.encode(chunks)


# print(time.time() - now)



In [16]:
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

In [1]:
from fastembed import TextEmbedding
import pymupdf
from semantic_text_splitter import TextSplitter
import time
import os
import numpy as np
splitter = TextSplitter(1000)
files = []
model = TextEmbedding(model_name = "sentence-transformers/all-MiniLM-L6-v2",
                      providers=["CUDAExecutionProvider", "CPUExecutionProvider"])

def fastembed():
# get all pdfs from test_files

  for file in os.listdir("bench"):

    text = []
    doc = pymupdf.open("bench/" + file)

    for page in doc:
        text.append(page.get_text())

    text = " ".join(text)
    chunks = splitter.chunks(text)
    embeddings = list(model.embed(chunks))

%timeit -n 20 fastembed()

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

2024-10-12 10:09:19.913 | ERROR    | fastembed.common.model_management:download_model:248 - Could not download model from HuggingFace: An error happened while trying to locate the file on the Hub and we cannot find the requested files in the local cache. Please check your connection and try again or make sure your Internet connection is on. Falling back to other sources.
100%|██████████| 83.2M/83.2M [01:34<00:00, 881kiB/s] 


NoSuchFile: [ONNXRuntimeError] : 3 : NO_SUCHFILE : Load model from /tmp/fastembed_cache/fast-all-MiniLM-L6-v2/model.onnx failed:Load model /tmp/fastembed_cache/fast-all-MiniLM-L6-v2/model.onnx failed. File doesn't exist

In [ ]:
from embed_anything import EmbeddingModel, WhichModel, embed_query, TextEmbedConfig
import os
import pymupdf
from semantic_text_splitter import TextSplitter

model = EmbeddingModel.from_pretrained_onnx(
    WhichModel.Bert, "AllMiniLML6V2"
)
splitter = TextSplitter(1000)
config = TextEmbedConfig(batch_size=256)

def embed_anything():
# get all pdfs from test_files

  for file in os.listdir("bench"):
    text = []
    doc = pymupdf.open("bench/" + file)

    for page in doc:
        text.append(page.get_text())
    
    text = " ".join(text)
    chunks = splitter.chunks(text)
    embed_query(chunks, model, config)

%timeit -n 20 embed_anything()

Session is using CUDAExecutionProvider
207 ms ± 6.01 ms per loop (mean ± std. dev. of 7 runs, 20 loops each)


In [5]:
from embed_anything import EmbeddingModel, WhichModel, embed_query, TextEmbedConfig
import os
import pymupdf
from semantic_text_splitter import TextSplitter
import os

model = EmbeddingModel.from_pretrained_hf(
    WhichModel.Bert, "sentence-transformers/all-MiniLM-L6-v2"
)
splitter = TextSplitter(1000)
config = TextEmbedConfig()

def embed_anything():
# get all pdfs from test_files

  for file in os.listdir("bench"):
    text = []
    doc = pymupdf.open("bench/" + file)

    for page in doc:
        text.append(page.get_text())
    
    text = " ".join(text)
    chunks = splitter.chunks(text)
    embeddings = embed_query(chunks, model, config)

%timeit -n 20 embed_anything()

Loading weights from "/home/akshay/.cache/huggingface/hub/models--sentence-transformers--all-MiniLM-L6-v2/snapshots/8b3219a92973c328a8e22fadcfa821b5dc75636a/model.safetensors"
1 s ± 19.4 ms per loop (mean ± std. dev. of 7 runs, 20 loops each)
